## Import libraries

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 50)

## Load datasets

A quick look into the dataset and we could notice that the special character _"\N"_ is used instead of _NaN_ for missing values.

In [3]:
# X1: entry dataset (note: X2 is the testing dataset)
# use row `Unamed: 0` as the row index
X1 = pd.read_csv("datasets/X1.csv", na_values="\\N")
X1.head()

,Unnamed: 0,title,img_url,description,ratings,n_votes,is_adult,production_year,runtime,genres,release_year,studio,img_embeddings,text_embeddings
0,2502,Letters to Juliet,https://m.media-amazon.com/images/M/MV5BMjg0OT...,Letters to Juliet: Directed by Gary Winick. Wi...,6.5,92937.0,0,2010,105.0,"Adventure,Comedy,Drama",2010.0,Sum.,"[0.25030804, 2.4058464, 1.0431569, 0.030648155...","[-0.6795498, 0.35658365, 0.9994932, -0.9793934..."
1,6238,Veil of Tears,https://m.media-amazon.com/images/M/MV5BZjMxOD...,Veil of Tears: Directed by William Gereghty. W...,7.9,11.0,0,1996,NaN,"Action,Crime,Drama",2014.0,WF,"[0.51250213, 2.8152602, 0.46308166, 0.29031387...","[-0.6202415, 0.31657028, 0.9992422, -0.9703722..."
2,1800,International Velvet,https://m.media-amazon.com/images/M/MV5BOGVkYj...,International Velvet: Directed by Bryan Forbes...,5.9,1345.0,0,1978,127.0,"Drama,Family,Sport",1978.0,MGM,"[0.18073043, 0.24735461, 0.63652813, 0.2496522...","[-0.709996, 0.4233521, 0.99980927, -0.98892415..."
3,2675,8 Seconds,https://m.media-amazon.com/images/M/MV5BYjY4Nz...,8 Seconds: Directed by John G. Avildsen. With ...,6.6,4851.0,0,1994,105.0,"Biography,Drama,Sport",1994.0,NL,"[0.025015268, 0.9105338, 0.3878257, 0.3421247,...","[-0.7416838, 0.38435012, 0.9998453, -0.9874693..."
4,3674,Penitentiary II,https://m.media-amazon.com/images/M/MV5BNjQyZW...,Penitentiary II: Directed by Jamaa Fanaka. Wit...,4.1,549.0,0,1982,108.0,"Crime,Drama,Sport",1982.0,MGM,"[0.19079691, 1.9068279, 0.29114372, 0.19527505...","[-0.65501904, 0.3845747, 0.9996712, -0.9766391..."


In [ ]:
#X1.loc[X1["title"] == "Clown"]

inputs dataset has dimension (3540, 14)

One first thing we can notice is that our dataset use a special character "\\N" for empty values. We should modify them to NaN.

In [4]:
# Y1: target dataset
Y1 = pd.read_csv("datasets/Y1.csv", header=None, names=["revenues"])
Y1.head()

,revenues
0,7.158530e+07
1,8.964834e+04
2,3.164907e+07
3,3.898197e+07
4,9.697023e+06


target dataset has dimension (3540, 1)

In [5]:
# X2: testing entry dataset
X2 = pd.read_csv("datasets/X2.csv", na_values="\\N")

In [6]:
print(f"X1 dataset contains {X1.shape[0]} observations and {X1.shape[1]} features")
print(f"X2 dataset (for prediction only) contains {X2.shape[0]} observations")

print(f"features: {list(X1.columns)}")
print(f"target: {list(Y1.columns)}")

X1 dataset contains 3540 observations and 14 features
X2 dataset (for prediction only) contains 1518 observations
features: ['Unnamed: 0', 'title', 'img_url', 'description', 'ratings', 'n_votes', 'is_adult', 'production_year', 'runtime', 'genres', 'release_year', 'studio', 'img_embeddings', 'text_embeddings']
target: ['revenues']


## Dataset description

inputs (X1):     
- `title`: title of the movie.    
- `ratings`: rating on IMDB.    
- `n_votes`: number of votes that are averaged for the given rating.    
- `is_adult`: is the movie destined for a mature audience (0 or 1).    
- `production_year`: the year the movie was produced.    
- `release_year`: the year the movie was released.    
- `runtime`: how long the movie lasts for (in minutes).    
- `genres`: a list of maximum 3 genres that fits the movie.   
- `studio`: the movie studio that produced the movie.        
- `img.url`: the url of the poster of the movie.    
- `img.embeddings`: vector of size 2048 representing the poster.    
- `description`: synopsis of the movie.    
- `text.embeddings`: vector of size 768 representing the synopsis.

There is also an `"Unnamed: 0"` column that seems to be an **id for the movie**. We can remove it.

target (Y1):     
- `revenue`: the amount in dollars the movie made in the USA.    

In [7]:
# drop unecessary column `Unnamed: 0`
X1.drop("Unnamed: 0", axis=1, inplace=True)

For feature engineering and the sake of simplicity, we're gonna concatenate the inputs `X1` with the target `Y1`

In [8]:
df = pd.concat([X1, Y1], axis = 1)
df.head()

,title,img_url,description,ratings,n_votes,is_adult,production_year,runtime,genres,release_year,studio,img_embeddings,text_embeddings,revenues
0,Letters to Juliet,https://m.media-amazon.com/images/M/MV5BMjg0OT...,Letters to Juliet: Directed by Gary Winick. Wi...,6.5,92937.0,0,2010,105.0,"Adventure,Comedy,Drama",2010.0,Sum.,"[0.25030804, 2.4058464, 1.0431569, 0.030648155...","[-0.6795498, 0.35658365, 0.9994932, -0.9793934...",7.158530e+07
1,Veil of Tears,https://m.media-amazon.com/images/M/MV5BZjMxOD...,Veil of Tears: Directed by William Gereghty. W...,7.9,11.0,0,1996,NaN,"Action,Crime,Drama",2014.0,WF,"[0.51250213, 2.8152602, 0.46308166, 0.29031387...","[-0.6202415, 0.31657028, 0.9992422, -0.9703722...",8.964834e+04
2,International Velvet,https://m.media-amazon.com/images/M/MV5BOGVkYj...,International Velvet: Directed by Bryan Forbes...,5.9,1345.0,0,1978,127.0,"Drama,Family,Sport",1978.0,MGM,"[0.18073043, 0.24735461, 0.63652813, 0.2496522...","[-0.709996, 0.4233521, 0.99980927, -0.98892415...",3.164907e+07
3,8 Seconds,https://m.media-amazon.com/images/M/MV5BYjY4Nz...,8 Seconds: Directed by John G. Avildsen. With ...,6.6,4851.0,0,1994,105.0,"Biography,Drama,Sport",1994.0,NL,"[0.025015268, 0.9105338, 0.3878257, 0.3421247,...","[-0.7416838, 0.38435012, 0.9998453, -0.9874693...",3.898197e+07
4,Penitentiary II,https://m.media-amazon.com/images/M/MV5BNjQyZW...,Penitentiary II: Directed by Jamaa Fanaka. Wit...,4.1,549.0,0,1982,108.0,"Crime,Drama,Sport",1982.0,MGM,"[0.19079691, 1.9068279, 0.29114372, 0.19527505...","[-0.65501904, 0.3845747, 0.9996712, -0.9766391...",9.697023e+06


## EDA

### Types of variables

Let's check the different types of variables

In [ ]:
# types of variables
df.dtypes.value_counts()

In [ ]:
df.info()

- `n_votes` and `release_year` are of type **float** but we could have thought they would be of type **int**.

In [ ]:
df["n_votes"].unique()

array([9.2937e+04, 1.1000e+01, 1.3450e+03, ..., 2.2860e+03, 4.1810e+03,
       2.7379e+04])

In [ ]:
df["release_year"].unique()

array([2010., 2014., 1978., 1994., 1982., 2000., 1990., 2002., 1997.,
       2009., 2001., 2003., 2007., 1992., 1998., 1995., 2008., 1983.,
       2015., 1991., 2016., 2006., 1973., 2004., 2011., 1999., 1986.,
       2005., 1996., 2013., 1993., 1989., 2012., 1987., 1988., 2017.,
       1980., 1985., 1981., 1979., 1984., 1977., 2018., 1946., 1975.,
       1966., 1971., 1974., 1941., 1957., 1970., 1976., 1972., 1959.])

For sure we could convert `release_year` to type **int**.

### Duplicated observations

Let's check if we have any duplicate observations (we saw before that there could be duplicated movies with different `movie_id`)

In [ ]:
df[df.duplicated(subset=df.columns.difference(["revenues"]))].count()

We have 432 duplicated observations

### Missing values

Let's see if there is any empty values

In [ ]:
df.isna().sum()

In [ ]:
# number of missing values
X2.isna().sum()

Dataset for modelling :
There are 264 missing values for `runtime` feature and 4 missing values for `genres` feature.

Dataset for prediction :
There are 106 missing values for `runtime` feature and 4 missing values for `genres` feature.

In [ ]:
# percentage of missing values
((df.isna().sum() / df.shape[0]) * 100).round(decimals = 2)

In [ ]:
((X2.isna().sum() / X2.shape[0]) * 100).round(decimals = 2)

In [ ]:
plt.figure(figsize = (20, 10))
sns.heatmap(df.isna(), cbar = False)

In [ ]:
df.loc[df.isna().any(axis=1)]

Let's check the rows containing missing values

**Rule of thumb**: _if values are missing at random and percentage of observations with these missing values are less than $5\%$. We can drop them without risking of creating bias in our dataset._

We have $0.3\%$ of entries with missing values for `genres` features. These are random missing values (no reason for these to be missing, probably forgotten) so we can definitelty drop these entries without risk of creating bias in our dataset.
However, for the `runtime` feature, we have ~ $7\%$ of missing values. That's a little bit much for removing all these entries even though they also seem to be random missing values.

We could try to impute by mean or something else.

In [ ]:
sns.histplot(df["runtime"])

print(df["runtime"].mode())

If data is missing randomly but the rows with these missing values are more than $5\%$ of the dataset, we can use **mean** (in case feature is normally distributed) or **median** (otherwise) imputation. We can also consider **mode** imputation.

However, keep in mind it affects data distribution (in particular the variance is reduced).

In [ ]:
print("runtime mean: {}".format(df["runtime"].mean()))
print("runtime median: {}".format(df["runtime"].median()))
print("runtime mode: {}".format(df["runtime"].mode()))

In [29]:
import numpy as np 

from itertools import chain

df.dropna(subset=["genres"], axis=0, inplace=True)

genres_list = list(chain(*df["genres"].str.split(",").tolist()))

unique_genres = []
genres_counter = {}

# retrieve each genre
for genre in genres_list:
	if genre in genres_counter:
		genres_counter[genre] += 1
	else:
		genres_counter[genre] = 1

for genre, count in genres_counter.items():
	genres_counter[genre] = np.round(count / len(genres_list), 3)

genres_counter

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit_transform(df["genres"])

mlb.classes_

array([',', '-', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'M', 'N', 'R', 'S',
       'T', 'V', 'W', 'a', 'c', 'd', 'e', 'g', 'h', 'i', 'l', 'm', 'n',
       'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y'], dtype=object)

### Analysis of features

In [ ]:
plt.subplots(figsize=(20,30))

i = 1

for col in df.select_dtypes("int"):
    plt.subplot(4,3,i)
    sns.histplot(df[col])
    i += 1

There seems to be only _non-adult_ movies (to confirm later).

Movies were mainly produced between **1990** and **2010**. We have a slightly left skewed distribution but it is more or less **normally distributed**.

In [ ]:
df["is_adult"].value_counts()

Indeed, we **do not have any movies** for a _mature audience_. 
Therefore, we could drop this column.

In [ ]:
plt.subplots(figsize=(20,30))

i = 1

for col in df.select_dtypes("float"):
    plt.subplot(4, 3, i)
    sns.histplot(df[col])
    i += 1

- `ratings`, `runtime` and `release_year` features are more or less **normally distributed**.

- `ratings` have a mean around **6.5**.

- Most movies were released between **2005** and **2010**. 

- `n_votes` feature and `revenues` target are **heavily right skewed**. We will have to manage that.

### Skewness and outliers analysis

In [ ]:
df.skew().sort_values(ascending=False)

In [ ]:
sns.boxplot(y = df["n_votes"])

In [ ]:
sns.boxplot(y = df["revenues"])

In [ ]:
sns.boxplot(y = np.log(df["n_votes"]))

In [ ]:
sns.histplot(np.log(df["n_votes"]))

In [ ]:
sns.boxplot(y = np.log(df["revenues"]))

In [ ]:
sns.histplot(np.log(df["revenues"]))

For these two variables, 50% of the data are concentrated on a small range of values. But they contain a lot of outliers until pretty high values (that's why distribution is heavily right skewed).

We see that we can perform a **log** transform on the 2 features to fix the skewness.

In [ ]:
df.describe()

Let's analyze the range of the different numericals features

In [ ]:
(df.select_dtypes(include=["int64", "float64"]).max() - df.select_dtypes(include=["int64", "float64"]).min()).round(decimals=3)

Finally we count the number of unique values for the 2 categorical features `genres` and `studio`.

In [ ]:
df["genres"].describe()

`genre` feature contains list of maximum 3 most representative genres for each movies so there are many differents list of genres which does not mean there a as much different genres. We should preprocess them before then count how many different genres there are. 
However, we can expect there shouldn't be too many differents.

In [ ]:
df["studio"].describe()

We see there are $498$ different studios (**high-cardinality** problem), therefore, it will result in a lof of features if we One-Hot encode them. As a consequence, we would explose the dimensionnality of the datas and there would be more risk to overfit (curse of dimensionnality). Better to Label encode ? Let's check first the distribution of this feature

In [ ]:
from collections import Counter

Counter(df["studio"]).most_common()

There are some studio that only appear one time. We could definitely remove them and replace them by a category called `other`. Then One-Hot encode this feature.

Or, we can also try "Count Encoding" that replaces each `studio` value with the number of times it appears in the dataset